In [3]:
from keras.models import Sequential
from keras.layers import Convolution2D, Dense , Dropout, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'



###########################################################################################################################


# Building the Convolutional Neural Network



model = Sequential()


#Convolution layer and pooling
model.add(Convolution2D(32, (3, 3), input_shape=(128, 128, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Convolution2D(32, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))


# Flattening the layers
model.add(Flatten())


# Fully connected layer
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.30))

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.30))

model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=27, activation='softmax'))     #Try removing it 


# Compile 
model.compile(
    'adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
) 

model.summary()




############################################################################################################################

#Augmentation of data


datagenerator_train = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        #zoom_range=0.2,
        #horizontal_flip=True
)

datagenerator_test = ImageDataGenerator(
        rescale=1./255
)

dataset_train = datagenerator_train.flow_from_directory(
        'data/train',
        target_size=(128, 128),
        batch_size=10,
        color_mode='grayscale',
        class_mode='categorical'
)

dataset_test = datagenerator_test.flow_from_directory(
        'data/test',
        target_size=(128 , 128),
        batch_size=10,
        color_mode='grayscale',
        class_mode='categorical'
)

model.fit(
        dataset_train,
        steps_per_epoch=12841,  # No of images in training set
        epochs=5,
        validation_data=dataset_train,
        validation_steps=4268   # No of images in test set
)



##################################################################################################################


# Saving CNN model


_json = model.to_json()
with open("model.json", "w") as _file:
    _file.write(_json)

model.save_weights('model.h5')

print('Model saved')


                                     


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               3686528   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)              